pd.DataFrmaeから1年単位の折れ線グラフを描く関数

In [ ]:
# 1年単位で折れ線グラフを並べて描く関数
def plot_year_linegraph(df, y, fig_x=18, label=None, loc="upper left", title=None, sharey=True):
    
    '''
    1年単位で折れ線グラフを描く関数
    縦の長さは年数に応じてよしなにやる
    
    df(pd.DataFrame): プロットしたいデータフレーム、datetime型のindexとintのyear列が入っている必要がある
    y(str): 描画したい変数名
    figsize(tuple): 全ての年を合わせたグラフの大きさ
    label(str): ラベル
    loc(str): ラベルの配置場所
    title(str): タイトル
    sharey(bool): y軸を揃えるかどうか
    '''
    

    from datetime import datetime, date, timedelta
    from matplotlib.dates import DateFormatter, MonthLocator
    import matplotlib.ticker as ticker
    
    df = df.sort_index()
    years = np.sort(df.year.unique())

    fig, axes = plt.subplots(nrows=years.shape[0], ncols=1, sharey=sharey, figsize=(fig_x, years.shape[0]*3))
    for i, year in enumerate(years):
        df_selected = df[df.year == year]
        axes[i].plot(df_selected.index, df_selected[y], label=label)
    
        # 表示形式の設定
        mformat = DateFormatter("%m月")
        axes[i].xaxis.set_major_formatter(mformat)
    
        # 区切りの設定
        mlocator = MonthLocator()
        axes[i].xaxis.set_major_locator(mlocator)
    
        # 上限下限の設定
        datemin = date(year, 1, 1)
        datemax = date(year + 1, 1, 1)
        axes[i].set_xlim(datemin, datemax)
    
        # ラベルの設定
        if label != None:
            axes[i].legend(loc=loc)
        
    # タイトルの設定
    if title != None:
        fig.suptitle(title)
    
    plt.show()
    return 

In [ ]:
# 標準化をする自作関数
def zscore(x, axis=None):
    '''
    np.arrayを標準化(平均0, 標準偏差1にする)関数
    np.arrayを入力してnp.arrayを返す
    
    x(np.array): 標準化するデータ
    axis(0 or 1): 標準化する軸、Noneはデータ全体、0は行方向、1は列方向に正規化する
    '''
    
    x_mean = np.mean(x, axis=axis, keepdims=True)
    x_std = np.std(x, axis=axis, keepdims=True) # keepdimsは返り値を(1, 1)のarrayにするかfloatにするか選ぶ
    zscore = (x - x_mean) / x_std
    return zscore

In [ ]:
# indexに2000年の日付、columnに年が入った移動平均のDFを作る関数
def get_nday_walking_mean(df, y, ndays=1, min_periods=1):
    '''
    indexにpd.Timestamp型の日付、columnに描画したい変数のみを入れたdfを入れると
    indexに2000年で同じ月日が入って、columnにyearが入るdfを返す関数
    
    df(pd.DataFrame): indexにpd.Timestamp型の日付、columnに描画したい変数が入ったdf
    y(str): 使用する変数名
    ndays(int): 何日移動平均を描画するか
    min_periods(int): 移動平均において非NAが何日以上あれば欠測値にしないか
    '''
    
    # dfを日付単位で行を整え、移動平均を求める
    df_allday = df.resample('D').mean().sort_index()
    df_allday['day'] = df_allday.index.day
    df_allday['month'] = df_allday.index.month
    df_allday['year'] = df_allday.index.year
    df_allday['walking_mean'] = df_allday[y].rolling(window=ndays, min_periods=min_periods, center=True).mean()
    
    # 月日でピボットする
    df_pivoted = df_allday.pivot_table(values='walking_mean', index=['month', 'day'], columns=['year'], dropna=False).reset_index(drop=False)
    
    # 2000年(閏年)のDFを作る
    index_2000 = pd.date_range('2000/01/01', periods=366)
    df_2000 = pd.DataFrame(index=index_2000)
    df_2000['month'] = df_2000.index.month
    df_2000['day'] = df_2000.index.day
    
    # 月日でマージする
    df_merged = pd.merge(df_2000, df_pivoted, how='left', on=['month', 'day'])
    
    # indexを振り直して月日の行を削除
    df_merged.index = index_2000
    df_merged = df_merged.drop(['month', 'day'], axis=1)
    
    return df_merged

In [ ]:
from matplotlib.dates import DateFormatter, MonthLocator


# get_nday_walking_meanで作ったDFを入れて、折れ線グラフを重ねて描画する関数
def plot_year_multiline(df, years = None, xlim=None, ylim=None, title=None, figsize=(12, 8)):
    '''
    indexに1年分の日付、columnに年、valueに描画したい値が入ったDFから折れ線グラフを重ねて描画する関数
    
    df(pd.DataFrame): indexに1年分の日付、columnに年、valueに描画したい値が入ったDF
    years(list): 描画したい年をintで入れたlist
    xlim(tuple): X軸の描画範囲を指定する、始点と終点にpd.Timestampが入ったtuple
    ylim(tuple): Y軸の描画範囲を指定する、始点と終点にint or floatが入ったtuple
    title(str): タイトル
    figsize(tuple): 描画の大きさを指定するtuple
    '''

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    
    if years is not None:
        [ax.plot(tmp.index, tmp.loc[:,column], label=column, alpha=0.5) for column in tmp.columns if column in years]
    else:
        [ax.plot(tmp.index, tmp.loc[:,column], label=column, alpha=0.5) for column in tmp.columns]

    # Formatterの設定
    mformatter = DateFormatter('%m月')

    # Locatorの設定
    mlocator = MonthLocator()
    
    # xlim, ylimの設定
    if xlim is not None:
        ax.set_xlim(xlim)
    
    if ylim is not None:
        ax.set_ylim(ylim)
        
    # タイトルの設定
    if title is not None:
        ax.set_title(title)

    ax.xaxis.set_major_formatter(mformatter)
    ax.xaxis.set_major_locator(mlocator)
    ax.legend()

    plt.show()
    return

In [ ]:
# indexに日付が入ったDFから曜日毎の箱ひげ図を描画する関数
def plot_boxplot_per_weekday(df, y, ylim=None, yticks=None, title=None, figsize=(12, 8)):
    '''
    indexに日付が入ったDFから任意の変数に対して曜日ごとの箱ひげ図を描く関数
    
    df(pd.DataFrame): 使用するDF、indexに日付が入っている必要がある
    y(str): 描画したい変数名
    ylim(tuple): 描画するyの範囲を指定したtuple
    yticks(tuple): 描画するyの区切りを指定したtuple, (始点、終点、区切り単位)
    title(str): タイトル
    figsize(tuple): 描画する大きさ
    '''
    
    # 曜日列の設定
    df_add_weekday = deepcopy(df)
    df_add_weekday['weekday'] = df_add_weekday.index.weekday
    
    # 曜日ラベルのlistを作成
    weekday_label = ["月曜", "火曜", "水曜", "木曜", "金曜", "土曜", "日曜"]
    
    # 曜日毎に値を代入したlistを束ねたlistを作る
    data_list = []
    for week_num, df_grouped in df.groupby('weekday'):
        data_list.append(df_grouped[y].tolist())
        
    # 描画
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    ax.boxplot(data_list)
    
    ax.set_xticklabels(weekday_label)
    if ylim is not None:
        ax.set_ylim(ylim)
    if yticks is not None:
        ax.set_yticks(np.arange(yticks))
    if title is not None:
        ax.set_title(title)
        
    return plt